Explore the eBird API

- https://documenter.getpostman.com/view/664302/S1ENwy59
- https://pypi.org/project/ebird-api/
- https://github.com/ProjectBabbler/ebird-api

In [1]:
from dotenv import load_dotenv
from ebird.api import get_observations
import os
import pandas
import json
from glob import glob
from pathlib import Path
from tqdm import tqdm 

load_dotenv()  # take environment variables from .env.
api_key = os.environ['EBIRD_API']

In [2]:
records = get_observations(api_key, 'L227544', back=7)
# records

In [3]:
from ebird.api import get_taxonomy, get_taxonomy_forms, get_taxonomy_versions

# Get all the species in the eBird taxonomy.
taxonomy = get_taxonomy(api_key)
taxonomy = pandas.DataFrame.from_records(taxonomy)
# taxonomy

In [4]:
def load_birdnet_log(path, confidence_threshold: float = 0.25) -> pandas.DataFrame:
    lines = [json.loads(s) for s in open(path)]
    raw = pandas.DataFrame.from_records(lines)

    raw = raw[raw['msg'] == 'success'].copy()
    raw = raw.explode(['results'])
    raw['timestamp'] = pandas.to_datetime(raw['timestamp'])

    raw[['name','confidence']] = pandas.DataFrame(raw.results.tolist(), index=raw.index)
    raw[['species', 'common']] = raw['name'].str.split("_", expand = True)
    raw.drop(columns=['msg', 'results', 'filename', 'oldest', 'name', 'skipped', 'hour_of_day'], inplace=True)
    return raw.query('confidence > @confidence_threshold', engine='python').copy()


In [5]:
def generate_ebird_record(df, path, location):
    species_by_minute = (
        df
        .assign(timestamp=lambda r: r['timestamp'].dt.round('min'))
        .groupby(by=['timestamp', 'common'])
        .agg({'species': 'count', 'confidence': 'max'})
        .reset_index()
        .rename(columns={0: "calls"})
    )
    species_by_minute

    obs = []
    for row in species_by_minute.itertuples():
        row = row._asdict()
        assert row['common'] in list(taxonomy.comName.values)
        
        common = row['common']
        if common =="Willie-wagtail": common = "Willie Wagtail"
        if common == "Eurasian Blackbird": common = "Common Blackbird"
        if "Gray" in common: common = common.replace("Gray", "Grey")

        obs.append([
            common,                                     # Common/scientific Name
            "",                                         # Genus
            "",                                         # Species
            "+",                                        # Species Count
            "",                                         # Species Comments
            location,                                   # Location Name
            "",                                         # latitude
            "",                                         # longitude
            row['timestamp'].strftime('%m/%d/%Y'),      # Observation date
            row['timestamp'].strftime('%H:%M'),         # Start time
            "",                                         # State
            "AU",                                       # Country
            "casual",                                   # Protocol
            "",                                         # Number of observers
            "1",                                        # Duration (minutes)
            "Y",                                        # All observations?
            "",                                         # Distance covered
            "",                                         # Area covered
            f"{row['species']} BirdNET calls; {row['confidence']} confidence", # Comments
        ])

    if len(obs):
        Path(path).parent.mkdir(exist_ok=True)
        with open(path, 'wt') as fp:
            for line in obs:
                fp.write(','.join(line) + '\n')


In [6]:
location = os.environ['EBIRD_LOCATION']

In [7]:
import shutil
shutil.rmtree('ebird')
for logpath in tqdm(glob("logs/*")):
    raw = load_birdnet_log(logpath, confidence_threshold=0.5)
    csvpath = logpath.replace("logs/", "ebird/") + ".csv"
    generate_ebird_record(raw, csvpath, location)
    

100%|██████████| 251/251 [02:30<00:00,  1.67it/s]
